In [12]:
import geopandas as gpd
import pandas as pd

In [2]:
data_pop=gpd.read_file('data/output/1_population_density/grid_pop_51_100m.gpkg')
data_degurba_grod=gpd.read_file('data/output/2_grid_degurba/degurba_5100.gpkg')

In [3]:
data_desa_raster=gpd.read_file('data/output/0_raster_data_desa/desa_pend_rasterize_51.gpkg')

In [7]:
pop_w_grid=data_pop.sjoin(data_degurba_grod)[['j_pend','p_builtup','classification','geometry']]

In [9]:
data_desa=data_desa_raster.sjoin(pop_w_grid)

In [10]:
data_desa

,iddesa,DN,id,TOTAL_PEND,geometry,index_right,j_pend,p_builtup,classification
0,5101010002,1.0,1,13066,"POLYGON ((11401400.000 -1013900.000, 11401800....",13908,3.663487,0.003637,RURAL
6,5101010008,7.0,7,3776,"POLYGON ((11402700.000 -1020000.000, 11402900....",13908,3.663487,0.003637,RURAL
0,5101010002,1.0,1,13066,"POLYGON ((11401400.000 -1013900.000, 11401800....",13907,0.000000,0.003637,RURAL
6,5101010008,7.0,7,3776,"POLYGON ((11402700.000 -1020000.000, 11402900....",13907,0.000000,0.003637,RURAL
0,5101010002,1.0,1,13066,"POLYGON ((11401400.000 -1013900.000, 11401800....",13910,0.000000,0.034227,CLUSTER
...,...,...,...,...,...,...,...,...,...
714,5171031011,715.0,715,18858,"POLYGON ((11468200.000 -1060800.000, 11468300....",300183,29.737134,0.062106,CENTRE
714,5171031011,715.0,715,18858,"POLYGON ((11468200.000 -1060800.000, 11468300....",300222,27.648852,0.062106,CENTRE
714,5171031011,715.0,715,18858,"POLYGON ((11468200.000 -1060800.000, 11468300....",300182,5.095408,0.062106,CENTRE
714,5171031011,715.0,715,18858,"POLYGON ((11468200.000 -1060800.000, 11468300....",300181,3.403899,0.062106,CENTRE


In [16]:
tab_data=pd.pivot_table(data_desa,index='iddesa',columns='classification',values='j_pend',aggfunc='sum').fillna(0).reset_index()

In [17]:
def define_LAU(y):
    total_pop=y.CENTRE+y.CLUSTER+y.RURAL
    if total_pop==0:
        return 'RURALS'
    elif (y.CENTRE/total_pop)>=0.5:
        return 'CITIES'
    elif (((y.CENTRE/total_pop)<0.5)&((y.RURAL/total_pop)<0.5)):
        return 'TOWNS'
    else:
        return 'RURALS'

In [18]:
tab_data['LAU']=tab_data.apply(lambda y:define_LAU(y),axis=1)

In [19]:
tab_data

classification,iddesa,CENTRE,CLUSTER,RURAL,LAU
0,5101010002,0.000000,17135.275753,2215.923816,TOWNS
1,5101010003,0.000000,7785.082680,799.405750,TOWNS
2,5101010004,0.000000,7793.398546,1930.204678,TOWNS
3,5101010005,0.000000,10357.611751,1940.690220,TOWNS
4,5101010006,0.000000,10508.136132,2164.228628,TOWNS
...,...,...,...,...,...
710,5171031007,33548.193942,0.000000,0.000000,CITIES
711,5171031008,25288.621075,0.000000,0.000000,CITIES
712,5171031009,45730.695304,0.000000,0.000000,CITIES
713,5171031010,18349.891582,0.000000,0.000000,CITIES


In [22]:
kdprov='51'
peta_desa=gpd.read_file(r"data/input/Master_Data/DESA_20201_BYINDO.gpkg").query('kdprov==@kdprov')

In [23]:
peta_desa_LAU=peta_desa.merge(tab_data[['iddesa','LAU']],how='left').fillna('RURALS')

In [25]:
peta_desa_LAU.to_file('data/output/3_small_spatial_unit_degurba/small_spatial_unit_'+kdprov+'.gpkg',driver='GPKG')